In [1]:
import glob
import cv2
import numpy as np
import re

In [2]:
def imgageSearch(address):
    img_name=glob.glob(str(address))

    class_name=[]

    pattern=re.compile(r'16BIT\d\d\d')

    for i in range(len(img_name)):
        match=re.findall(pattern,img_name[i])
        class_name.append(match)
        
    class_name=np.array(class_name)
    totalNoOfImages=len(img_name)
    class_name=class_name.reshape(totalNoOfImages,)
    img=[]

    for i in img_name:
        c=cv2.imread(i,0)
        img.append(c)
    img=np.array(img)

    return class_name , img

In [3]:
address='C:/Users/All/Desktop/Face Recognition/train/*.*'

class_name , img = imgageSearch(address)

In [4]:
def faceDetection(class_name, img):
    face_cascade = cv2.CascadeClassifier('C:/Users/ALL/AppData/Local/Programs/Python/Python37/Scripts/haarcascade_frontalface_default.xml')
    faces=[]
    classes=[]

    for i in range(len(img)):
        t=face_cascade.detectMultiScale(img[i], 1.2, 5)
        if type(t) != tuple:
            if t.shape != (1,4):
                (x,y,h,w) = t[0]
                faces.append(img[i][y:y+w , x:x+h])
                classes.append(class_name[i])
            if t.shape == (1,4):
                for (x,y,h,w) in t:
                    faces.append(img[i][y:y+w , x:x+h])
                    classes.append(class_name[i])
    faces=np.array(faces)
    classes=np.array(classes)


    for i in range(len(faces)):
        faces[i]=cv2.resize(faces[i],(64,64))
    
    return classes , faces

In [5]:
classes , faces = faceDetection(class_name,img)

In [6]:
def vectorFace(faces):
    vfaces=[]
    
    for i in range(len(faces)):
        vfaces.append(faces[i].ravel())
    
    vfaces=np.array(vfaces)
    
    return vfaces.T

In [7]:
vfaces = vectorFace(faces)

In [8]:
mean=vfaces.mean(axis=1)


In [9]:
mfaces=[]
(p,n)=vfaces.shape
for i in range(n):
    mfaces.append(vfaces[:,i]-mean)
mfaces=np.array(mfaces)
mfaces=mfaces.T

In [10]:
j=0
co=np.zeros(1)

for i in range(len(classes)):
    if(i==0):
        b=classes[i]
        co[j]+=1
    else:
        a=classes[i]
        if(b==a):
            co[j]+=1

        else:
            b=a
            j+=1
            co=np.append(co,np.zeros(1))
            co[j]+=1


In [11]:
S=mfaces.T@mfaces


In [12]:
wt,vt = np.linalg.eig(S)

In [13]:
order=np.argsort(wt)
fliporder=np.flip(order)
svt=vt[:,fliporder[:len(co)]]


In [14]:
ef=mfaces@svt

In [15]:
Z=(ef.T)@(mfaces)

In [16]:
Z_mean=(Z.mean(axis=1))

In [17]:
Z_mean_class = []
count=0
for i in range(len(co)):
    Z_mean_class.append(np.zeros((len(co))))
    for j in range(int(co[i])):
        Z_mean_class[int(i)]=Z_mean_class[int(i)]+Z[:,int(count)]
        count+=1
    Z_mean_class[i]=Z_mean_class[i]/co[i]
Z_mean_class=np.array(Z_mean_class)


In [18]:
Sb=np.zeros((len(co),len(co)))

for i in range(len(co)):
    ans=(((Z_mean_class[i]-Z_mean).reshape(len(co),1))@(((Z_mean_class[i]-Z_mean).reshape(len(co),1)).T))
    Sb=Sb+ans

In [19]:
Sw=np.zeros((len(co),len(co)))
count=0
for i in range(len(co)):
    for j in range(int(co[i])) :
        ans=((Z[:,count]-Z_mean_class[i]).reshape(len(co),1))@((Z[:,count]-Z_mean_class[i]).reshape(len(co),1).T)
        Sw=Sw+ans
        count+=1

In [20]:
va , vec = np.linalg.eig((np.linalg.inv(Sw))@Sb)

In [21]:
weights=vec.T@Z

In [22]:
address='C:/Users/All/Desktop/Face Recognition/TEST_FINAL_/*.*'

class_test , imgt = imgageSearch(address)

classest , facest = faceDetection(class_test,imgt)

if(len(classest)!=len(class_test)):
    print("Few test images face weren't detected by haar cascade")

In [23]:
vfacest = vectorFace(facest)

In [24]:
mfacest=[]
(p,n)=vfacest.shape
for i in range(n):
    mfacest.append(vfacest[:,i]-mean)
mfacest=np.array(mfacest)
mfacest=mfacest.T

In [25]:
Zt=ef.T@mfacest

In [26]:
weightst=vec.T@Zt

In [27]:
minans=99999999
index=[]
(p,d) = weights.shape
(k,l) = weightst.shape
for i in range(l):
    minans=99999999
    index.append(0)
    for j in range(d):
        pqr=np.linalg.norm(weightst[:,i]-weights[:,j])
        if(pqr<minans):
            minans = pqr
            index[i] = j

In [28]:
correct=0
for i in range(l):
    print("Class :-",classest[i])
    print("prediction :-",classes[index[i]])
    if(classest[i]==classes[index[i]]):
        correct+=1
print("Accuracy :-",(correct/l)*100)

Class :- 16BIT015
prediction :- 16BIT015
Class :- 16BIT018
prediction :- 16BIT018
Class :- 16BIT019
prediction :- 16BIT019
Class :- 16BIT025
prediction :- 16BIT025
Class :- 16BIT029
prediction :- 16BIT029
Class :- 16BIT031
prediction :- 16BIT031
Class :- 16BIT036
prediction :- 16BIT036
Class :- 16BIT038
prediction :- 16BIT038
Class :- 16BIT039
prediction :- 16BIT039
Class :- 16BIT045
prediction :- 16BIT045
Class :- 16BIT049
prediction :- 16BIT049
Class :- 16BIT051
prediction :- 16BIT051
Class :- 16BIT052
prediction :- 16BIT052
Class :- 16BIT053
prediction :- 16BIT028
Class :- 16BIT054
prediction :- 16BIT054
Class :- 16BIT055
prediction :- 16BIT055
Class :- 16BIT056
prediction :- 16BIT056
Class :- 16BIT057
prediction :- 16BIT057
Class :- 16BIT058
prediction :- 16BIT058
Class :- 16BIT059
prediction :- 16BIT059
Class :- 16BIT060
prediction :- 16BIT060
Accuracy :- 95.23809523809523
